<a href="https://colab.research.google.com/github/Palaeoprot/Bear/blob/main/14_06_25_PhASTM_RegExDistiller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Biopython --quiet

In [ ]:
# ===== Cell 1 =====
# Setup

import json
import pandas as pd
import requests
import time
from pathlib import Path
from typing import List, Dict, Optional, Any
from IPython.display import display
import glob
from datetime import datetime

# Mount Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive mounted successfully.")
except ImportError:
    print("⚠️ Google Colab not detected. Using local paths.")

# Load tissue dictionary from JSON file
TISSUES_JSON_PATH = "/content/drive/MyDrive/Colab_Notebooks/DICTIONARIES/GeneFamily/collagen/TISSUES.json"

try:
    with open(TISSUES_JSON_PATH, 'r') as f:
        TISSUE_GENE_MAPPING = json.load(f)

    available_tissues = list(TISSUE_GENE_MAPPING.keys())
    print(f"✅ Loaded tissue dictionary with {len(available_tissues)} tissue types")

except FileNotFoundError:
    print(f"⚠️ Tissue dictionary not found at: {TISSUES_JSON_PATH}")
    print("   Using fallback tissue options")
    available_tissues = ["COL1", "Bone", "Cartilage", "Skin", "Tendon"]
    TISSUE_GENE_MAPPING = {"COL1": ["COL1A1", "COL1A2"]}

except json.JSONDecodeError as e:
    print(f"⚠️ Error reading JSON file: {e}")
    print("   Using fallback tissue options")
    available_tissues = ["COL1", "Bone", "Cartilage", "Skin", "Tendon"]
    TISSUE_GENE_MAPPING = {"COL1": ["COL1A1", "COL1A2"]}



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully.
✅ Loaded tissue dictionary with 51 tissue types


# PhASTM RegEx Distiller: Automated Gene Family Exon Discovery

This notebook implements a dynamic discovery workflow to automatically find all orthologous genes in a gene family using Ensembl Gene Trees, retrieve their exon structures, and create a standardized mapping system that solves the "split exon" problem.

## Workflow:
1. **Input**: Ensembl Gene Tree ID + Reference Species
2. **Discovery**: Automatically find all orthologues in the gene family
3. **Extraction**: Retrieve detailed exon structure for every orthologue
4. **Standardization**: Map all exons to reference species numbering
5. **Output**: Clean, standardized dataset ready for PhASTM pipeline

# 🧬 Ensembl Alignment Names Explained

The `ensembl_alignment_names` refer to **multiple genome alignments** provided by **Ensembl Compara**, used for comparative genomics. Each alignment includes a set of species aligned together using a specific alignment pipeline.

Below is a breakdown of each available alignment name, what it means, and when to use it.

## 🔍 Alignment Descriptions

| Name | Species Count | Taxonomic Group | Tool Used | Notes |
|------|---------------|------------------|-----------|-------|
| `"10 primates EPO"` | ~10 | Primates | EPO | High-quality primate alignment including major species like human, chimp, gorilla |
| `"24 primates EPO-Extended"` | ~24 | Primates | EPO | More primate species than above, slightly lower quality |
| `"26 rodents Cactus"` | ~26 | Rodents | Cactus | Includes mice, rats, squirrels, voles, etc. |
| `"31 primates Cactus"` | ~31 | Primates | Cactus | Even more primates; uses the Cactus aligner which scales well |
| `"44 eutherian mammals EPO"` | ~44 | Placental mammals | EPO | Broad alignment covering many mammalian orders |
| `"60 amniota vertebrates Mercator-Pecan"` | ~60 | Amniotes | Mercator-Pecan | Includes mammals, birds, reptiles, some amphibians |
| `"91 eutherian mammals EPO-Extended"` | ~91 | Placental mammals | EPO | Most comprehensive placental mammal alignment |
| `"104 fish EPO_LOW_COVERAGE"` | ~104 | Fish | EPO | Many fish species but low coverage due to divergence |

## 🧰 Alignment Tools Overview

### EPO (Enredo-Pecan-Ortheus)
A high-quality pipeline combining:
- **Enredo**: Detects rearrangements  
- **Pecan**: Performs global multiple alignment  
- **Ortheus**: Infers ancestral sequences  

Best for **closely related species** with good reference genomes.

### Cactus
Designed for **large-scale and highly divergent alignments**. Scales better than EPO but may be less sensitive in some regions.

### Mercator-Pecan
Older alignment strategy using **Mercator** for synteny mapping and **Pecan** for alignment. Less commonly used now, but still exists for historical datasets.

## 📌 Choosing the Right Alignment

Use these guidelines to choose the right alignment based on your research goals:

- ✅ **Looking for high-quality primate alignments?**  
  Use `"10 primates EPO"` or `"24 primates EPO-Extended"`

- ✅ **Studying rodent evolution or diversity?**  
  Use `"26 rodents Cactus"`

- ✅ **Broad mammalian comparisons (e.g., humans, dogs, cows)?**  
  Use `"44 eutherian mammals EPO"` or `"91 eutherian mammals EPO-Extended"`

- ✅ **Need vertebrate-wide comparisons (mammals + birds + reptiles)?**  
  Use `"60 amniota vertebrates Mercator-Pecan"`

- ⚠️ **Working with fish genomes?**  
  Use `"104 fish EPO_LOW_COVERAGE"` cautiously — covers many species, but only conserved regions are reliably aligned.

In [ ]:
#@markdown ### **🎯 Discovery Method Selection**
discovery_method = "gene_symbol" #@param ["gene_tree_direct", "gene_symbol"]

#@markdown ### **🔬 Species Filtering Options**
species_filtering = "manual_limit" #@param ["no_filtering", "alignment_subset", "manual_limit"]

#@markdown **If using alignment_subset, select alignment:**
ensembl_alignment = "44 eutherian mammals EPO" #@param ["10 primates EPO", "24 primates EPO-Extended", "26 rodents Cactus", "31 primates Cactus", "44 eutherian mammals EPO", "60 amniota vertebrates Mercator-Pecan", "91 eutherian mammals EPO-Extended", "104 fish EPO_LOW_COVERAGE"] {allow-input: true}

#@markdown **If using manual_limit, set max species:**
max_species_limit = 50 #@param {type:"slider", min:5, max:300, step:5}

#@markdown ### **🧬 Gene Configuration**
gene_tree_id = "ENSGT00940000156584" #@param ["ENSGT00940000156584", "ENSGT00940000162105"] {allow-input: true}
gene_symbol = "COL1A1" #@param ["COL1A1", "COL1A2"] {allow-input: true}
reference_species = "homo_sapiens" #@param {type:"string"}

#@markdown ### **🏥 Tissue Context**
tissue_of_interest = "COL1" #@param {type:"string"}

#@markdown ### **💾 Output Configuration**
output_base_path = '/content/drive/MyDrive/Colab_Notebooks/PhASTM_Output' #@param {type:"string"}

#@markdown ### **🔄 Pipeline Control**
force_rerun = False #@param {type:"boolean"}

# Create output directory
output_dir = Path(output_base_path)
output_dir.mkdir(parents=True, exist_ok=True)

# Validate tissue selection and get associated genes
if tissue_of_interest in TISSUE_GENE_MAPPING:
    selected_genes = TISSUE_GENE_MAPPING[tissue_of_interest]
    print(f"✅ Valid tissue selection: {tissue_of_interest}")
else:
    print(f"⚠️ Tissue '{tissue_of_interest}' not found in dictionary")
    print(f"📋 Available tissues: {', '.join(list(TISSUE_GENE_MAPPING.keys())[:10])}...")
    tissue_of_interest = list(TISSUE_GENE_MAPPING.keys())[0]
    selected_genes = TISSUE_GENE_MAPPING[tissue_of_interest]
    print(f"🔄 Using fallback tissue: {tissue_of_interest}")



In [ ]:
# Create output directory
output_dir = Path(output_base_path)
output_dir.mkdir(parents=True, exist_ok=True)

# Validate tissue selection and get associated genes
if tissue_of_interest in TISSUE_GENE_MAPPING:
    selected_genes = TISSUE_GENE_MAPPING[tissue_of_interest]
    print(f"✅ Valid tissue selection: {tissue_of_interest}")
else:
    print(f"⚠️ Tissue '{tissue_of_interest}' not found in dictionary")
    print(f"📋 Available tissues: {', '.join(list(TISSUE_GENE_MAPPING.keys())[:10])}...")
    tissue_of_interest = list(TISSUE_GENE_MAPPING.keys())[0]
    selected_genes = TISSUE_GENE_MAPPING[tissue_of_interest]
    print(f"🔄 Using fallback tissue: {tissue_of_interest}")

# ===== FILE EXISTENCE CHECK =====
def generate_expected_filename(gene: str, tissue: str, filtering: str, alignment: str = None, limit: int = None) -> str:
    """Generate the expected filename based on current parameters."""
    if filtering == "no_filtering":
        return f"{gene}_{tissue}_all_species_raw_exons_*.csv"
    elif filtering == "alignment_subset":
        alignment_short = alignment.replace(" ", "_").replace("-", "_")
        return f"{gene}_{tissue}_{alignment_short}_raw_exons_*.csv"
    elif filtering == "manual_limit":
        return f"{gene}_{tissue}_manual_{limit}_species_raw_exons_*.csv"
    else:
        return f"{gene}_{tissue}_*_raw_exons_*.csv"

def check_existing_files() -> List[Path]:
    """Check for existing files matching current parameters."""
    pattern = generate_expected_filename(gene_symbol, tissue_of_interest, species_filtering,
                                       ensembl_alignment, max_species_limit)
    search_pattern = str(output_dir / pattern)
    existing_files = [Path(f) for f in glob.glob(search_pattern)]
    return sorted(existing_files, key=lambda x: x.stat().st_mtime, reverse=True)  # Most recent first

# Check for existing files
existing_files = check_existing_files()
should_run_pipeline = True

if existing_files and not force_rerun:
    print(f"\n🔍 EXISTING FILES FOUND!")
    print(f"="*50)

    for i, file_path in enumerate(existing_files[:5]):  # Show up to 5 most recent
        # Get file info
        stat = file_path.stat()
        file_size = stat.st_size / (1024*1024)  # MB
        mod_time = datetime.fromtimestamp(stat.st_mtime)

        print(f"{i+1}. {file_path.name}")
        print(f"   📅 Created: {mod_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"   📁 Size: {file_size:.1f} MB")

        # Try to get row count
        try:
            df_info = pd.read_csv(file_path, nrows=0)  # Just get columns
            df_sample = pd.read_csv(file_path, nrows=1000)  # Sample to estimate
            estimated_rows = len(df_sample) * (file_size / (df_sample.memory_usage(deep=True).sum() / (1024*1024)))
            print(f"   📊 Estimated rows: ~{int(estimated_rows):,}")
        except:
            print(f"   📊 Could not read file info")
        print()

    if len(existing_files) > 5:
        print(f"   ... and {len(existing_files) - 5} more files")

    print(f"🤔 Do you want to use existing data or re-run the pipeline?")
    print(f"   ✅ RECOMMENDED: Use existing data (faster)")
    print(f"   🔄 Alternative: Set force_rerun=True to generate new data")
    print(f"   ⚠️  Re-running will take time and API calls")

    should_run_pipeline = False

    # Set a global variable for other cells to check
    most_recent_file = existing_files[0]
    print(f"\n💡 To use existing data in Cell 5:")
    print(f"   raw_exon_data = pd.read_csv('{most_recent_file}')")

elif force_rerun and existing_files:
    print(f"\n🔄 FORCE RERUN ENABLED")
    print(f"   Found {len(existing_files)} existing files but will run pipeline anyway")
    should_run_pipeline = True

else:
    print(f"\n📭 No existing files found matching current parameters")
    should_run_pipeline = True

print(f"\n✅ Configuration Complete:")
print(f"   - Discovery Method: {discovery_method}")
print(f"   - Species Filtering: {species_filtering}")

if species_filtering == "no_filtering":
    print(f"   - Will use ALL species in gene tree (could be 200+)")
elif species_filtering == "alignment_subset":
    print(f"   - Selected Alignment: {ensembl_alignment}")
    print(f"   - Expected Species Count: ~{ensembl_alignment.split()[0]}")
elif species_filtering == "manual_limit":
    print(f"   - Manual Limit: First {max_species_limit} species from gene tree")

print(f"   - Gene Symbol: {gene_symbol}")
print(f"   - Reference Species: {reference_species}")
print(f"   - Tissue Context: {tissue_of_interest}")
print(f"   - Output Directory: {output_dir}")
print(f"   - Pipeline Will Run: {'✅ YES' if should_run_pipeline else '❌ NO (use existing data)'}")

# Store configuration for other cells
pipeline_config = {
    'should_run': should_run_pipeline,
    'existing_files': existing_files,
    'most_recent_file': existing_files[0] if existing_files else None
}

✅ Valid tissue selection: COL1

📭 No existing files found matching current parameters

✅ Configuration Complete:
   - Discovery Method: gene_symbol
   - Species Filtering: no_filtering
   - Will use ALL species in gene tree (could be 200+)
   - Gene Symbol: COL1A1
   - Reference Species: homo_sapiens
   - Tissue Context: COL1
   - Output Directory: /content/drive/MyDrive/Colab_Notebooks/PhASTM_Output
   - Pipeline Will Run: ✅ YES


In [ ]:
# ===== Cell 3  =====
# API functions for tree member extraction

ENSEMBL_SERVER = "https://rest.ensembl.org"

def debug_api_response(url: str, response_data: dict) -> None:
    """Debug helper to understand API response structure."""
    print(f"🔍 API Response Debug for: {url}")
    print(f"   Top-level keys: {list(response_data.keys()) if isinstance(response_data, dict) else 'Not a dict'}")
    if isinstance(response_data, dict):
        for key, value in response_data.items():
            if isinstance(value, list):
                print(f"   {key}: List with {len(value)} items")
            elif isinstance(value, dict):
                print(f"   {key}: Dict with keys {list(value.keys())}")
            else:
                print(f"   {key}: {type(value).__name__} = {value}")

def debug_tree_structure(node, depth=0, max_depth=3):
    """Debug helper to examine tree structure in detail."""
    indent = "  " * depth
    if depth > max_depth:
        print(f"{indent}... (max depth reached)")
        return

    if isinstance(node, dict):
        print(f"{indent}Node keys: {list(node.keys())}")
        for key, value in node.items():
            if key == 'children' and isinstance(value, list):
                print(f"{indent}{key}: List with {len(value)} children")
                for i, child in enumerate(value[:2]):  # Show first 2 children
                    print(f"{indent}  Child {i}:")
                    debug_tree_structure(child, depth + 2, max_depth)
            elif key in ['id', 'species', 'sequence']:
                print(f"{indent}{key}: {value}")

def extract_members_from_tree_structure(tree_node: Dict) -> List[Dict]:
    """
    CORRECTED: Extract all gene members from Ensembl tree structure.
    Fixed based on actual tree structure analysis.
    """
    members = []

    def traverse_tree(node, depth=0):
        if not isinstance(node, dict):
            return

        # Check if this is a leaf node (gene) based on actual structure
        node_id = node.get('id', {})
        node_sequence = node.get('sequence', {})
        node_taxonomy = node.get('taxonomy', {})

        # A gene node has: id, sequence, and taxonomy (but not children, or minimal children)
        has_sequence = bool(node_sequence)
        has_id = bool(node_id)
        has_taxonomy = bool(node_taxonomy)
        has_children = 'children' in node and node['children']

        # This is a gene if it has sequence and id
        if has_sequence and has_id:
            # Extract gene ID
            if isinstance(node_id, dict):
                stable_id = node_id.get('accession')
                source = node_id.get('source', 'EnsEMBL')
            else:
                stable_id = str(node_id)
                source = 'EnsEMBL'

            # Extract species from taxonomy
            species_name = None
            if isinstance(node_taxonomy, dict):
                species_name = (node_taxonomy.get('scientific_name') or
                              node_taxonomy.get('name') or
                              node_taxonomy.get('common_name'))

            # Extract gene name from sequence if available
            gene_name = stable_id
            if isinstance(node_sequence, dict) and 'name' in node_sequence:
                gene_name = node_sequence['name']
            elif 'name' in node:
                gene_name = node['name']

            # Only add if we have essential information
            if stable_id and stable_id.startswith('ENS'):  # Valid Ensembl ID
                members.append({
                    'gene_stable_id': stable_id,
                    'species': species_name or 'unknown_species',
                    'display_name': gene_name,
                    'source': source,
                    'has_sequence': has_sequence,
                    'location': node_sequence.get('location') if has_sequence else None
                })

                if depth < 3:
                    print(f"{'  ' * depth}✅ Found gene: {stable_id} ({species_name})")

        # Continue traversing children
        if 'children' in node and isinstance(node['children'], list):
            for child in node['children']:
                traverse_tree(child, depth + 1)

    print(f"🔍 Starting corrected tree traversal...")
    traverse_tree(tree_node)
    print(f"✅ Corrected traversal complete. Found {len(members)} members.")
    return members

def get_gene_tree_by_symbol(symbol: str, species: str) -> Optional[Dict]:
    """
    Get gene tree using gene symbol with enhanced debugging.
    """
    print(f"🔍 Looking up gene tree for {symbol} in {species}")
    ext = f"/genetree/member/symbol/{species}/{symbol}"
    try:
        r = requests.get(ENSEMBL_SERVER + ext, headers={"Content-Type": "application/json"}, timeout=30)
        r.raise_for_status()
        tree_data = r.json()

        # Debug the response structure
        debug_api_response(ENSEMBL_SERVER + ext, tree_data)

        # Get tree ID from top level
        tree_id = tree_data.get('id', 'Unknown')

        # The actual tree structure is under 'tree' key
        tree_structure = tree_data.get('tree', {})

        print(f"\n🔍 Detailed tree structure analysis:")
        debug_tree_structure(tree_structure, max_depth=2)

        # Extract members recursively from tree structure
        print(f"\n🔍 Extracting members from tree structure...")
        members = extract_members_from_tree_structure(tree_structure)

        print(f"  ✅ Found tree {tree_id} with {len(members)} members")

        # If no members found, let's try a different approach
        if not members:
            print(f"  🔍 No members found with standard extraction. Trying alternative methods...")
            # Try extracting from root level
            alt_members = extract_members_from_tree_structure(tree_data)
            if alt_members:
                print(f"  ✅ Alternative extraction found {len(alt_members)} members")
                members = alt_members

        return {'tree_id': tree_id, 'members': members, 'full_response': tree_data}

    except requests.exceptions.RequestException as e:
        print(f"  ❌ Error: {e}")
        return None

def test_with_known_working_tree() -> Optional[Dict]:
    """
    Test with a known working gene tree ID with enhanced debugging.
    """
    print("🧪 Testing with known working tree ID from Ensembl docs...")
    known_tree_id = "ENSGT00390000003602"
    ext = f"/genetree/id/{known_tree_id}"
    try:
        r = requests.get(ENSEMBL_SERVER + ext, headers={"Content-Type": "application/json"}, timeout=30)
        r.raise_for_status()
        tree_data = r.json()

        debug_api_response(ENSEMBL_SERVER + ext, tree_data)

        # For direct tree lookup, the tree data might be at root level
        if 'tree' in tree_data:
            tree_id = tree_data.get('id', known_tree_id)
            tree_structure = tree_data.get('tree', {})
        else:
            tree_id = tree_data.get('id', known_tree_id)
            tree_structure = tree_data

        print(f"\n🔍 Detailed tree structure analysis:")
        debug_tree_structure(tree_structure, max_depth=2)

        # Extract members from the tree structure
        print(f"\n🔍 Extracting members from tree structure...")
        members = extract_members_from_tree_structure(tree_structure)

        print(f"  ✅ Known tree test: {len(members)} members found")
        return {'tree_id': tree_id, 'members': members, 'full_response': tree_data}

    except Exception as e:
        print(f"  ❌ Known tree test failed: {e}")
        return None

def get_exon_details_for_gene(gene_id: str, species: str) -> Optional[List[Dict]]:
    """Get exon details for a specific gene."""
    print(f"    📍 Fetching exons: {gene_id} ({species})")
    ext = f"/lookup/id/{gene_id}?expand=1"
    try:
        time.sleep(0.1)  # Rate limiting
        r = requests.get(ENSEMBL_SERVER + ext, headers={"Content-Type": "application/json"}, timeout=30)
        r.raise_for_status()
        gene_data = r.json()

        if 'Transcript' in gene_data and gene_data['Transcript']:
            # Use canonical transcript if available
            transcript = next((t for t in gene_data['Transcript'] if t.get('is_canonical', False)),
                            gene_data['Transcript'][0])
            exons = transcript.get('Exon', [])
            print(f"      ✅ Found {len(exons)} exons")
            return exons
        return None
    except requests.exceptions.RequestException:
        print(f"      ⚠️ Could not fetch exons for {gene_id}")
        return None

print("✅ API functions loaded with enhanced debugging and corrected member extraction")

✅ API functions loaded with enhanced debugging and corrected member extraction


In [ ]:
# ===== Cell 4 =====
# Enhanced pipeline with clear species filtering options


# Check if pipeline should run
if 'pipeline_config' in globals() and not pipeline_config['should_run']:
    print("⏭️  SKIPPING PIPELINE EXECUTION")
    print("   Existing data found and force_rerun=False")
    print("   To use existing data:")

    if pipeline_config['most_recent_file']:
        print(f"   raw_exon_data = pd.read_csv('{pipeline_config['most_recent_file']}')")
        # Actually load the data
        raw_exon_data = pd.read_csv(pipeline_config['most_recent_file'])
        print(f"   ✅ Loaded {len(raw_exon_data)} exons from {raw_exon_data['species'].nunique()} species")

        # Display quick summary
        print("\n--- Loaded Data Summary ---")
        display(raw_exon_data[['species', 'gene_name', 'exon_number_in_gene', 'start', 'end', 'length']].head(5))

    print("   To force re-run: Set force_rerun=True in Cell 2")

else:
    print("🚀 RUNNING PIPELINE - No existing data found or force_rerun=True")

def apply_species_filtering(members: List[Dict], filtering_method: str) -> List[Dict]:
    """
    Apply species filtering based on user selection with clear logic.
    """
    original_count = len(members)

    if filtering_method == "no_filtering":
        print(f"\n📊 No filtering applied - using all {original_count} members")
        print(f"   ⚠️  This will process ALL species in the gene tree")
        return members

    elif filtering_method == "manual_limit":
        print(f"\n✂️ Applying manual species limit: {max_species_limit}")

        # Group by species and limit
        species_groups = {}
        for member in members:
            species = member.get('species', 'unknown')
            if species not in species_groups:
                species_groups[species] = []
            species_groups[species].append(member)

        # Take first N species
        selected_species = list(species_groups.keys())[:max_species_limit]
        filtered_members = []
        for species in selected_species:
            filtered_members.extend(species_groups[species])

        print(f"   📊 Filtered from {original_count} members ({len(species_groups)} species)")
        print(f"   📊 to {len(filtered_members)} members ({len(selected_species)} species)")
        return filtered_members

    elif filtering_method == "alignment_subset":
        print(f"\n🔬 Applying alignment-based filtering: {ensembl_alignment}")

        # Try the corrected alignment filtering
        try:
            filtered_members = filter_members_by_alignment_corrected(members, ensembl_alignment)

            if len(filtered_members) < 5:
                print(f"   ⚠️  Alignment filtering returned too few members ({len(filtered_members)})")
                print(f"   🔄 Falling back to manual limit of {max_species_limit}")
                return apply_species_filtering(members, "manual_limit")

            return filtered_members

        except Exception as e:
            print(f"   ❌ Alignment filtering failed: {e}")
            print(f"   🔄 Falling back to manual limit of {max_species_limit}")
            return apply_species_filtering(members, "manual_limit")

    else:
        print(f"   ❌ Unknown filtering method: {filtering_method}")
        print(f"   🔄 Falling back to manual limit of 50")
        return members[:50]

def run_comprehensive_gene_discovery_final() -> pd.DataFrame:
    """
    Final version with clear species filtering options.
    """
    print("\n" + "="*70)
    print("🚀 GENE FAMILY DISCOVERY WITH CONFIGURABLE FILTERING")
    print("="*70)

    tree_result = None

    # Strategy 1: Gene symbol lookup
    print(f"\n🎯 Strategy 1: Gene Symbol Lookup ({gene_symbol})")
    tree_result = get_gene_tree_by_symbol(gene_symbol, reference_species)

    # Fallback strategies (same as before)
    if not tree_result or not tree_result.get('members'):
        print(f"\n🧪 Strategy 2: Test with Known Working Tree")
        tree_result = test_with_known_working_tree()

    if not tree_result or not tree_result.get('members'):
        print(f"\n🔄 Strategy 3: Alternative Gene Symbols")
        alternative_genes = ["BRCA2", "COL1A2", "TP53"]
        for alt_gene in alternative_genes:
            print(f"  Trying {alt_gene}...")
            tree_result = get_gene_tree_by_symbol(alt_gene, reference_species)
            if tree_result and tree_result.get('members'):
                print(f"  ✅ Success with {alt_gene}!")
                break

    if not tree_result or not tree_result.get('members'):
        print("❌ All discovery strategies failed")
        return pd.DataFrame()

    # Apply user-selected filtering
    original_members = tree_result['members']
    filtered_members = apply_species_filtering(original_members, species_filtering)

    tree_result['members'] = filtered_members
    tree_result['tree_id'] = f"{tree_result['tree_id']}_{species_filtering}"

    members = tree_result['members']
    tree_id = tree_result['tree_id']

    print(f"\n✅ Successfully prepared gene family dataset!")
    print(f"   Tree ID: {tree_id}")
    print(f"   Members to process: {len(members)}")
    print(f"   Filtering method: {species_filtering}")

    # Show species distribution
    species_counts = {}
    for member in members:
        species = member.get('species', 'unknown')
        species_counts[species] = species_counts.get(species, 0) + 1

    print(f"\n📊 Final Species Distribution:")
    for i, (species, count) in enumerate(sorted(species_counts.items())[:15]):
        print(f"   {i+1:2d}. {species}: {count} genes")
    if len(species_counts) > 15:
        print(f"   ... and {len(species_counts) - 15} more species")

    # Continue with exon collection (same as before)
    print(f"\n📊 Collecting exon data for {len(members)} genes...")
    all_exon_data = []
    successful_genes = 0

    for i, member in enumerate(members, 1):
        gene_id = (member.get('gene_stable_id') or
                  member.get('protein_stable_id') or
                  member.get('id') or
                  member.get('stable_id'))

        species = (member.get('species') or
                  member.get('genome'))

        gene_name = member.get('display_name', member.get('name', 'Unknown'))

        if not gene_id or not species:
            continue

        print(f"  [{i:2d}/{len(members)}] Processing: {gene_id} ({species})")

        exons = get_exon_details_for_gene(gene_id, species)
        if exons:
            successful_genes += 1
            for exon_num, exon in enumerate(exons, 1):
                all_exon_data.append({
                    'species': species,
                    'gene_id': gene_id,
                    'gene_name': gene_name,
                    'exon_id': exon.get('id'),
                    'exon_number_in_gene': exon_num,
                    'start': exon.get('start'),
                    'end': exon.get('end'),
                    'strand': exon.get('strand'),
                    'length': (exon.get('end', 0) - exon.get('start', 0) + 1) if exon.get('start') and exon.get('end') else 0,
                    'tree_id': tree_id,
                    'filtering_method': species_filtering
                })

    if not all_exon_data:
        print("❌ No exon data could be collected")
        return pd.DataFrame()

    results_df = pd.DataFrame(all_exon_data)

    print(f"\n🎉 SUCCESS!")
    print(f"   📊 Total exons: {len(results_df)}")
    print(f"   🧬 Unique genes: {results_df['gene_id'].nunique()}")
    print(f"   🌍 Species: {results_df['species'].nunique()}")
    print(f"   🔬 Filtering: {species_filtering}")
    print("="*70)

    return results_df

# Execute the final pipeline
print("🚀 Starting gene family discovery with configurable filtering...")
raw_exon_data = run_comprehensive_gene_discovery_final()

# Display results (same as before)
if not raw_exon_data.empty:
    print("\n--- Sample of Successfully Collected Data ---")
    display(raw_exon_data[['species', 'gene_name', 'exon_number_in_gene', 'start', 'end', 'length']].head(10))

    print("\n--- Species Distribution ---")
    species_summary = raw_exon_data.groupby('species').agg({
        'gene_id': 'nunique',
        'exon_id': 'count',
        'length': 'mean'
    }).rename(columns={'gene_id': 'genes', 'exon_id': 'exons', 'length': 'avg_exon_length'})
    species_summary['avg_exon_length'] = species_summary['avg_exon_length'].round(1)
    display(species_summary.head(15))

🚀 RUNNING PIPELINE - No existing data found or force_rerun=True
🚀 Starting gene family discovery with configurable filtering...

🚀 GENE FAMILY DISCOVERY WITH CONFIGURABLE FILTERING

🎯 Strategy 1: Gene Symbol Lookup (COL1A1)
🔍 Looking up gene tree for COL1A1 in homo_sapiens
🔍 API Response Debug for: https://rest.ensembl.org/genetree/member/symbol/homo_sapiens/COL1A1
   Top-level keys: ['rooted', 'type', 'id', 'tree']
   rooted: int = 1
   type: str = gene tree
   id: str = ENSGT00940000156584
   tree: Dict with keys ['branch_length', 'taxonomy', 'events', 'confidence', 'children']

🔍 Detailed tree structure analysis:
Node keys: ['branch_length', 'taxonomy', 'events', 'confidence', 'children']
children: List with 2 children
  Child 0:
    Node keys: ['branch_length', 'children', 'confidence', 'events', 'taxonomy']
    children: List with 2 children
      Child 0:
        ... (max depth reached)
      Child 1:
        ... (max depth reached)
  Child 1:
    Node keys: ['children', 'events'

,species,gene_name,exon_number_in_gene,start,end,length
0,Poecilia formosa,ENSPFOG00000017127,1,413227,413308,82
1,Poecilia formosa,ENSPFOG00000017127,2,414910,415125,216
2,Poecilia formosa,ENSPFOG00000017127,3,415388,415410,23
3,Poecilia formosa,ENSPFOG00000017127,4,415720,415755,36
4,Poecilia formosa,ENSPFOG00000017127,5,416848,416940,93
5,Poecilia formosa,ENSPFOG00000017127,6,418338,418406,69
6,Poecilia formosa,ENSPFOG00000017127,7,418497,418541,45
7,Poecilia formosa,ENSPFOG00000017127,8,419122,419175,54
8,Poecilia formosa,ENSPFOG00000017127,9,419252,419305,54
9,Poecilia formosa,ENSPFOG00000017127,10,419414,419467,54



--- Species Distribution ---


,genes,exons,avg_exon_length
species,,,
Acanthochromis polyacanthus,3,122,97.8
Ailuropoda melanoleuca reference isolate,1,51,91.8
Amphilophus citrinellus,2,76,103.3
Amphiprion ocellaris ecotype Okinawa,2,100,111.1
Amphiprion percula,2,83,96.3
Anabas testudineus reference strain,2,100,111.1
Anas platyrhynchos platyrhynchos,1,9,174.7
Anolis carolinensis reference strain,1,51,91.4
Anser brachyrhynchus,1,51,85.3


In [ ]:
# ===== SAVE RESULTS =====
# Save the raw exon data with descriptive filenames based on filtering method

if not raw_exon_data.empty:
    # Create timestamp for unique filename
    timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")

    # Create descriptive filename based on filtering method
    if species_filtering == "no_filtering":
        filename = f"{gene_symbol}_{tissue_of_interest}_all_species_raw_exons_{timestamp}.csv"
        summary_suffix = "all_species"
    elif species_filtering == "alignment_subset":
        alignment_short = ensembl_alignment.replace(" ", "_").replace("-", "_")
        filename = f"{gene_symbol}_{tissue_of_interest}_{alignment_short}_raw_exons_{timestamp}.csv"
        summary_suffix = alignment_short
    elif species_filtering == "manual_limit":
        filename = f"{gene_symbol}_{tissue_of_interest}_manual_{max_species_limit}_species_raw_exons_{timestamp}.csv"
        summary_suffix = f"manual_{max_species_limit}_species"
    else:
        filename = f"{gene_symbol}_{tissue_of_interest}_unknown_filter_raw_exons_{timestamp}.csv"
        summary_suffix = "unknown_filter"

    # Save to CSV
    output_path = output_dir / filename
    raw_exon_data.to_csv(output_path, index=False)

    print(f"\n💾 RAW DATA SAVED SUCCESSFULLY!")
    print(f"   📁 File: {filename}")
    print(f"   📍 Location: {output_path}")
    print(f"   📊 Data: {len(raw_exon_data)} exons from {raw_exon_data['species'].nunique()} species")
    print(f"   🔬 Filtering: {species_filtering}")

    # Enhanced summary report
    summary_filename = f"{gene_symbol}_{tissue_of_interest}_{summary_suffix}_summary_{timestamp}.txt"
    summary_path = output_dir / summary_filename

    with open(summary_path, 'w') as f:
        f.write(f"PhASTM Gene Family Exon Discovery Summary\n")
        f.write(f"="*60 + "\n\n")
        f.write(f"Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Gene Symbol: {gene_symbol}\n")
        f.write(f"Tissue Context: {tissue_of_interest}\n")
        f.write(f"Species Filtering Method: {species_filtering}\n")

        if species_filtering == "alignment_subset":
            f.write(f"Alignment Used: {ensembl_alignment}\n")
        elif species_filtering == "manual_limit":
            f.write(f"Species Limit: {max_species_limit}\n")

        f.write(f"Discovery Method: {discovery_method}\n")
        f.write(f"Reference Species: {reference_species}\n")

        if 'tree_id' in raw_exon_data.columns:
            tree_id = raw_exon_data['tree_id'].iloc[0]
            f.write(f"Tree ID: {tree_id}\n")

        f.write(f"\nResults Summary:\n")
        f.write(f"- Data File: {filename}\n")
        f.write(f"- Total Exons: {len(raw_exon_data):,}\n")
        f.write(f"- Unique Genes: {raw_exon_data['gene_id'].nunique()}\n")
        f.write(f"- Species Count: {raw_exon_data['species'].nunique()}\n")
        f.write(f"- Average Exon Length: {raw_exon_data['length'].mean():.1f} bp\n")
        f.write(f"- Median Exon Length: {raw_exon_data['length'].median():.1f} bp\n")
        f.write(f"- Total Sequence Length: {raw_exon_data['length'].sum():,} bp\n")

        # Species distribution
        f.write(f"\nSpecies Distribution:\n")
        species_summary = raw_exon_data.groupby('species').agg({
            'gene_id': 'nunique',
            'exon_id': 'count',
            'length': ['mean', 'sum']
        })

        species_summary.columns = ['genes', 'exons', 'avg_length', 'total_length']
        species_summary = species_summary.sort_values('exons', ascending=False)

        for species, row in species_summary.head(20).iterrows():
            f.write(f"- {species}: {row['genes']} genes, {row['exons']} exons, ")
            f.write(f"avg {row['avg_length']:.1f} bp, total {row['total_length']:,} bp\n")

        if len(species_summary) > 20:
            f.write(f"... and {len(species_summary) - 20} more species\n")

        # Configuration details for reproducibility
        f.write(f"\nConfiguration (for reproducibility):\n")
        f.write(f"- gene_symbol = '{gene_symbol}'\n")
        f.write(f"- tissue_of_interest = '{tissue_of_interest}'\n")
        f.write(f"- species_filtering = '{species_filtering}'\n")
        f.write(f"- reference_species = '{reference_species}'\n")

        if species_filtering == "alignment_subset":
            f.write(f"- ensembl_alignment = '{ensembl_alignment}'\n")
        elif species_filtering == "manual_limit":
            f.write(f"- max_species_limit = {max_species_limit}\n")

    print(f"   📄 Summary: {summary_filename}")
    print(f"\n🎯 NEXT STEPS:")
    print(f"   1. Run Cell 5 for exon standardization")
    print(f"   2. Or load this data later with:")
    print(f"      raw_exon_data = pd.read_csv('{output_path}')")

    # Store the filename for easy access
    latest_data_file = output_path

else:
    print("❌ No data to save")


💾 RAW DATA SAVED SUCCESSFULLY!
   📁 File: COL1A1_COL1_all_species_raw_exons_20250614_164758.csv
   📍 Location: /content/drive/MyDrive/Colab_Notebooks/PhASTM_Output/COL1A1_COL1_all_species_raw_exons_20250614_164758.csv
   📊 Data: 4441 exons from 169 species
   🔬 Filtering: no_filtering
   📄 Summary: COL1A1_COL1_all_species_summary_20250614_164758.txt

🎯 NEXT STEPS:
   1. Run Cell 5 for exon standardization
   2. Or load this data later with:
      raw_exon_data = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/PhASTM_Output/COL1A1_COL1_all_species_raw_exons_20250614_164758.csv')


In [ ]:
# ===== Cell 5 =====
# Enhanced exon standardization with better reference selection and error handling

def find_best_reference_species(df: pd.DataFrame, preferred_species: str = "homo_sapiens") -> str:
    """Find the best reference species from available data."""
    species_counts = df.groupby('species')['exon_id'].count().sort_values(ascending=False)

    # Try preferred species first
    if preferred_species in species_counts.index:
        print(f"✅ Using preferred reference species: {preferred_species}")
        return preferred_species

    # Fallback to species with most exons
    best_species = species_counts.index[0]
    print(f"🔄 Preferred species '{preferred_species}' not found. Using {best_species} as reference")
    return best_species

def map_exons_to_reference_enhanced(target_exons: List[Dict], reference_exons: List[Dict],
                                   target_info: str) -> List[Dict]:
    """
    Enhanced exon mapping with better error handling and logging.
    Maps target species exons to reference species exon numbering.
    Handles split exons (5-6) and insertions (21a, 21b).
    """
    print(f"    🎯 Mapping {len(target_exons)} exons for {target_info}")

    mapped_results = []
    last_mapped_ref = 0
    insertion_counter = 0

    # Filter out exons without valid coordinates
    target_sorted = sorted([e for e in target_exons if e.get('start') and e.get('end')],
                          key=lambda x: x.get('start', 0))
    ref_sorted = sorted([e for e in reference_exons if e.get('start') and e.get('end')],
                       key=lambda x: x.get('start', 0))

    if not target_sorted or not ref_sorted:
        print(f"      ⚠️ Insufficient coordinate data for mapping")
        return []

    # Add reference numbers for easier lookup
    for i, ref_exon in enumerate(ref_sorted):
        ref_exon['ref_number'] = i + 1

    overlaps_found = 0
    insertions_found = 0

    for target_exon in target_sorted:
        target_start = target_exon.get('start', 0)
        target_end = target_exon.get('end', 0)

        # Find overlapping reference exons
        overlapping_refs = []
        for ref_exon in ref_sorted:
            ref_start = ref_exon.get('start', 0)
            ref_end = ref_exon.get('end', 0)

            # Check for overlap
            if max(target_start, ref_start) <= min(target_end, ref_end):
                overlapping_refs.append(ref_exon['ref_number'])

        # Determine standardized number
        if overlapping_refs:
            overlaps_found += 1
            if len(overlapping_refs) == 1:
                std_number = str(overlapping_refs[0])
            else:
                std_number = f"{min(overlapping_refs)}-{max(overlapping_refs)}"
            last_mapped_ref = max(overlapping_refs)
            insertion_counter = 0
        else:
            # This is an insertion relative to reference
            insertions_found += 1
            insertion_counter += 1
            std_number = f"{last_mapped_ref}{chr(96 + insertion_counter)}"  # 21a, 21b, etc.

        mapped_results.append({
            **target_exon,
            'standardized_exon_number': std_number,
            'mapping_type': 'mapped' if overlapping_refs else 'insertion'
        })

    print(f"      ✅ Mapped: {overlaps_found}, Insertions: {insertions_found}")
    return mapped_results

# Perform the enhanced standardization
if not raw_exon_data.empty:
    print("\n" + "="*70)
    print("🎯 ENHANCED EXON STANDARDIZATION")
    print("="*70)

    # Find best reference species (prefer homo_sapiens, fallback to most exons)
    ref_species = find_best_reference_species(raw_exon_data, reference_species)
    ref_exons = raw_exon_data[raw_exon_data['species'] == ref_species].to_dict('records')

    if not ref_exons:
        print("❌ No reference exons available for standardization")
    else:
        print(f"📐 Reference dataset: {len(ref_exons)} exons from {ref_species}")

        # Show reference gene info
        ref_genes = raw_exon_data[raw_exon_data['species'] == ref_species]['gene_id'].nunique()
        print(f"📊 Reference includes {ref_genes} genes from {ref_species}")

        all_standardized = []
        successful_mappings = 0
        failed_mappings = 0

        # Process each species-gene combination
        print(f"\n🔄 Processing exon mappings...")
        for (species, gene_id), group in raw_exon_data.groupby(['species', 'gene_id']):
            target_exons = group.to_dict('records')
            target_info = f"{species} ({gene_id})"

            mapped_exons = map_exons_to_reference_enhanced(target_exons, ref_exons, target_info)
            if mapped_exons:
                all_standardized.extend(mapped_exons)
                successful_mappings += 1
            else:
                failed_mappings += 1

        if all_standardized:
            standardized_df = pd.DataFrame(all_standardized)

            # Calculate detailed statistics
            total_exons = len(standardized_df)
            split_exons = len(standardized_df[standardized_df['standardized_exon_number'].str.contains('-', na=False)])
            insertion_exons = len(standardized_df[standardized_df['mapping_type'] == 'insertion'])
            mapped_exons = total_exons - insertion_exons

            print(f"\n🎉 STANDARDIZATION COMPLETED!")
            print(f"   📊 Total exons processed: {total_exons}")
            print(f"   ✅ Successfully mapped: {mapped_exons}")
            print(f"   🔄 Split exons detected: {split_exons}")
            print(f"   ➕ Novel insertions: {insertion_exons}")
            print(f"   🧬 Successful gene mappings: {successful_mappings}")
            print(f"   ❌ Failed gene mappings: {failed_mappings}")

            # Save the results with better filename
            timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
            if dataset_scope == "alignment_subset":
                alignment_short = ensembl_alignment.replace(" ", "_").replace("-", "_")
                output_filename = f"{gene_symbol}_{tissue_of_interest}_{alignment_short}_standardized_{timestamp}.csv"
            else:
                output_filename = f"{gene_symbol}_{tissue_of_interest}_all_species_standardized_{timestamp}.csv"

            output_path = output_dir / output_filename
            standardized_df.to_csv(output_path, index=False)

            print(f"\n💾 Results saved to: {output_path}")

            # Display sample results
            print("\n--- Sample of Standardized Results ---")
            display_columns = ['species', 'gene_name', 'exon_number_in_gene',
                             'standardized_exon_number', 'mapping_type', 'start', 'end', 'length']
            display(standardized_df[display_columns].head(20))

            # Summary by species
            print("\n--- Standardization Summary by Species ---")
            summary = standardized_df.groupby('species').agg({
                'exon_id': 'count',
                'mapping_type': lambda x: (x == 'mapped').sum(),
                'standardized_exon_number': lambda x: x.str.contains('-', na=False).sum()
            }).rename(columns={
                'exon_id': 'total_exons',
                'mapping_type': 'mapped_exons',
                'standardized_exon_number': 'split_exons'
            })
            summary['insertion_exons'] = summary['total_exons'] - summary['mapped_exons']
            display(summary.head(15))

            print(f"\n✅ Dataset ready for PhASTM pipeline!")
            print(f"   📁 File: {output_filename}")
            print(f"   📊 Coverage: {standardized_df['species'].nunique()} species, {standardized_df['gene_id'].nunique()} genes")
            print(f"   🎯 Reference species: {ref_species}")
            print("="*70)

        else:
            print("❌ No exons could be successfully standardized")
            print("💡 This might be due to:")
            print("   - Coordinate systems not being comparable")
            print("   - Different genome assemblies")
            print("   - Invalid exon coordinate data")

else:
    print("⚠️ Skipping standardization - no raw data available")


🎯 ENHANCED EXON STANDARDIZATION
🔄 Preferred species 'homo_sapiens' not found. Using Notamacropus eugenii as reference
📐 Reference dataset: 77 exons from Notamacropus eugenii
📊 Reference includes 1 genes from Notamacropus eugenii

🔄 Processing exon mappings...
    🎯 Mapping 27 exons for Acanthochromis polyacanthus (ENSAPOG00000005021)
      ✅ Mapped: 0, Insertions: 27
    🎯 Mapping 26 exons for Ailuropoda melanoleuca reference isolate (ENSAMEG00000009390)
      ✅ Mapped: 0, Insertions: 26
    🎯 Mapping 15 exons for Amphilophus citrinellus (ENSACIG00000004324)
      ✅ Mapped: 0, Insertions: 15
    🎯 Mapping 13 exons for Amphiprion ocellaris ecotype Okinawa (ENSAOCG00000009260)
      ✅ Mapped: 0, Insertions: 13
    🎯 Mapping 18 exons for Anabas testudineus reference strain (ENSATEG00000012756)
      ✅ Mapped: 0, Insertions: 18
    🎯 Mapping 26 exons for Anas platyrhynchos platyrhynchos (ENSAPLG00000007774)
      ✅ Mapped: 0, Insertions: 26
    🎯 Mapping 22 exons for Anolis carolinensis r

,species,gene_name,exon_number_in_gene,standardized_exon_number,mapping_type,start,end,length
0,Acanthochromis polyacanthus,brca2-201,27,0a,insertion,224036,224392,357
1,Acanthochromis polyacanthus,brca2-201,26,0b,insertion,224516,224626,111
2,Acanthochromis polyacanthus,brca2-201,25,0c,insertion,225130,225386,257
3,Acanthochromis polyacanthus,brca2-201,24,0d,insertion,225468,225606,139
4,Acanthochromis polyacanthus,brca2-201,23,0e,insertion,225728,225915,188
5,Acanthochromis polyacanthus,brca2-201,22,0f,insertion,226001,226181,181
6,Acanthochromis polyacanthus,brca2-201,21,0g,insertion,226273,226341,69
7,Acanthochromis polyacanthus,brca2-201,20,0h,insertion,226477,226510,34
8,Acanthochromis polyacanthus,brca2-201,19,0i,insertion,226568,226585,18
9,Acanthochromis polyacanthus,brca2-201,18,0j,insertion,226685,226842,158



--- Standardization Summary by Species ---


,total_exons,mapped_exons,split_exons,insertion_exons
species,,,,
Acanthochromis polyacanthus,27,0,0,27
Ailuropoda melanoleuca reference isolate,26,0,0,26
Amphilophus citrinellus,15,0,0,15
Amphiprion ocellaris ecotype Okinawa,13,0,0,13
Anabas testudineus reference strain,18,0,0,18
Anas platyrhynchos platyrhynchos,26,0,0,26
Anolis carolinensis reference strain,22,0,0,22
Aotus nancymaae,26,0,0,26
Aquila chrysaetos chrysaetos,25,0,0,25



✅ Dataset ready for PhASTM pipeline!
   📁 File: COL1A1_COL1_44_eutherian_mammals_EPO_standardized_20250614_162518.csv
   📊 Coverage: 169 species, 172 genes
   🎯 Reference species: Notamacropus eugenii


In [ ]:
# # ===== Cell 4 - CORRECTED =====
# # Enhanced pipeline with WORKING species filtering

# def get_alignment_species_list(alignment_name: str) -> Optional[List[str]]:
#     """
#     Get list of species in a specific Ensembl genomic alignment.
#     CORRECTED: Uses the proper API endpoints and alignment identification.
#     """
#     print(f"🔍 Fetching species list for alignment: {alignment_name}")

#     # Extract method from alignment name
#     if "EPO" in alignment_name:
#         method = "EPO"
#     elif "Cactus" in alignment_name:
#         method = "CACTUS"
#     elif "Mercator" in alignment_name or "Pecan" in alignment_name:
#         method = "PECAN"
#     else:
#         print(f"  ⚠️ Unknown alignment method in: {alignment_name}")
#         return None

#     print(f"  🔍 Using method: {method}")

#     try:
#         # Step 1: Get all species sets for this method
#         ext = f"/info/compara/species_sets/{method}"
#         r = requests.get(ENSEMBL_SERVER + ext, headers={"Content-Type": "application/json"}, timeout=30)
#         r.raise_for_status()
#         species_sets = r.json()

#         print(f"  📊 Found {len(species_sets)} species sets for {method}")

#         # Step 2: Find the set that matches our alignment description
#         target_set = None

#         # Extract expected species count from alignment name
#         expected_count = None
#         for word in alignment_name.split():
#             if word.isdigit():
#                 expected_count = int(word)
#                 break

#         print(f"  🎯 Looking for set with ~{expected_count} species")

#         # Find best matching set
#         best_match = None
#         best_score = 0

#         for species_set in species_sets:
#             if not isinstance(species_set, dict):
#                 continue

#             set_name = species_set.get('name', '').lower()
#             species_list = species_set.get('species', [])
#             species_count = len(species_list)

#             # Score this set based on name matching and species count
#             score = 0

#             # Name matching
#             alignment_words = alignment_name.lower().split()
#             for word in alignment_words:
#                 if word in set_name:
#                     score += 1

#             # Species count matching
#             if expected_count and abs(species_count - expected_count) <= 5:
#                 score += 2

#             # Prefer larger sets for mammals/eutherian
#             if "mammal" in alignment_name.lower() or "eutherian" in alignment_name.lower():
#                 if species_count >= 30:
#                     score += 1

#             print(f"    Set: {set_name} ({species_count} species) - Score: {score}")

#             if score > best_score:
#                 best_score = score
#                 best_match = species_set

#         if best_match:
#             species_list = [sp.get('name') for sp in best_match.get('species', []) if sp.get('name')]
#             print(f"  ✅ Selected: {best_match.get('name')} with {len(species_list)} species")
#             return species_list
#         else:
#             print(f"  ⚠️ No matching species set found")
#             return None

#     except requests.exceptions.RequestException as e:
#         print(f"  ❌ Error fetching species sets: {e}")
#         return None

# def get_mammalian_species_fallback() -> List[str]:
#     """
#     Fallback list of common mammalian species names as they appear in Ensembl.
#     Based on typical gene tree member species.
#     """
#     return [
#         'homo_sapiens', 'pan_troglodytes', 'gorilla_gorilla', 'pongo_abelii',
#         'macaca_mulatta', 'chlorocebus_sabaeus', 'papio_anubis',
#         'mus_musculus', 'rattus_norvegicus', 'cricetulus_griseus',
#         'bos_taurus', 'sus_scrofa', 'equus_caballus', 'canis_lupus_familiaris',
#         'felis_catus', 'ovis_aries', 'capra_hircus',
#         'monodelphis_domestica', 'ornithorhynchus_anatinus',
#         'gallus_gallus', 'xenopus_tropicalis', 'danio_rerio'
#     ]

# def filter_members_by_alignment_corrected(members: List[Dict], alignment_name: str) -> List[Dict]:
#     """
#     CORRECTED: Filter gene tree members using improved species matching.
#     """
#     print(f"\n🔬 Applying species filtering for: {alignment_name}")

#     # Try to get alignment species list
#     alignment_species = get_alignment_species_list(alignment_name)

#     if not alignment_species:
#         print(f"  🔄 API approach failed, using smart fallback...")

#         # Smart fallback based on alignment description
#         if "mammal" in alignment_name.lower() or "eutherian" in alignment_name.lower():
#             alignment_species = get_mammalian_species_fallback()
#             print(f"  📊 Using mammalian fallback: {len(alignment_species)} species")
#         elif "primate" in alignment_name.lower():
#             alignment_species = ['homo_sapiens', 'pan_troglodytes', 'gorilla_gorilla',
#                                'pongo_abelii', 'macaca_mulatta', 'chlorocebus_sabaeus', 'papio_anubis']
#             print(f"  📊 Using primate fallback: {len(alignment_species)} species")
#         elif "rodent" in alignment_name.lower():
#             alignment_species = ['mus_musculus', 'rattus_norvegicus', 'cricetulus_griseus']
#             print(f"  📊 Using rodent fallback: {len(alignment_species)} species")
#         else:
#             # Last resort: limit to a reasonable number
#             print(f"  🔄 Using size-based filter...")
#             expected_count = 44  # Default for your alignment
#             for word in alignment_name.split():
#                 if word.isdigit():
#                     expected_count = int(word)
#                     break

#             # Just take the first N members
#             filtered_members = members[:expected_count]
#             print(f"  📊 Limited to first {len(filtered_members)} members")
#             return filtered_members

#     # Apply species filtering
#     print(f"  🎯 Filtering against {len(alignment_species)} target species...")

#     # Get unique species in the members list
#     member_species = set(m.get('species', '').lower().replace(' ', '_') for m in members)
#     print(f"  📋 Available species in gene tree: {len(member_species)}")

#     # Show some examples for debugging
#     print(f"  🔍 Sample member species: {list(member_species)[:5]}")
#     print(f"  🔍 Sample target species: {alignment_species[:5]}")

#     # Try multiple matching strategies
#     filtered_members = []

#     # Strategy 1: Exact match
#     alignment_species_lower = [sp.lower().replace(' ', '_') for sp in alignment_species]
#     for member in members:
#         member_species = member.get('species', '').lower().replace(' ', '_')
#         if member_species in alignment_species_lower:
#             filtered_members.append(member)

#     print(f"  ✅ Exact matches: {len(filtered_members)}")

#     # Strategy 2: Partial matching if exact matching gives too few results
#     if len(filtered_members) < 10:
#         print(f"  🔄 Trying partial matching...")
#         filtered_members = []

#         for member in members:
#             member_species = member.get('species', '').lower()
#             for target_species in alignment_species:
#                 target_lower = target_species.lower()
#                 # Check if species names have significant overlap
#                 if any(word in member_species for word in target_lower.split('_') if len(word) > 3):
#                     filtered_members.append(member)
#                     break

#         print(f"  ✅ Partial matches: {len(filtered_members)}")

#     # Strategy 3: If still too few, just limit by count
#     if len(filtered_members) < 5:
#         print(f"  🔄 Filtering failed, using count limit...")
#         expected_count = 44
#         for word in alignment_name.split():
#             if word.isdigit():
#                 expected_count = int(word)
#                 break
#         filtered_members = members[:expected_count]

#     print(f"  📊 Final result: {len(filtered_members)} members from {len(set(m.get('species') for m in filtered_members))} species")

#     return filtered_members

In [ ]:
# # ===== Cell 4 =====
# # Enhanced pipeline with species filtering based on genomic alignments

# def get_alignment_species_list(alignment_name: str) -> Optional[List[str]]:
#     """
#     Get list of species in a specific Ensembl genomic alignment.
#     This will limit our gene tree results to just these species.
#     """
#     print(f"🔍 Fetching species list for alignment: {alignment_name}")

#     # Convert alignment name to API format
#     alignment_api_name = alignment_name.replace(" ", "%20")
#     ext = f"/info/compara/species_sets/{alignment_api_name}"

#     try:
#         r = requests.get(ENSEMBL_SERVER + ext, headers={"Content-Type": "application/json"}, timeout=30)
#         r.raise_for_status()
#         alignment_data = r.json()

#         if isinstance(alignment_data, list) and alignment_data:
#             species_list = [species.get('name') for species in alignment_data if species.get('name')]
#             print(f"  ✅ Found {len(species_list)} species in {alignment_name}")
#             return species_list
#         else:
#             print(f"  ⚠️ No species data found for {alignment_name}")
#             return None

#     except requests.exceptions.RequestException as e:
#         print(f"  ❌ Error fetching alignment species: {e}")
#         print(f"  🔄 Will use fallback species filtering")
#         return None

# def filter_members_by_alignment(members: List[Dict], alignment_species: Optional[List[str]]) -> List[Dict]:
#     """
#     Filter gene tree members to include only species from the selected alignment.
#     """
#     if not alignment_species:
#         print(f"  📊 No species filter applied - using all {len(members)} members")
#         return members

#     # Convert species names to lowercase for matching
#     alignment_species_lower = [sp.lower().replace(' ', '_') for sp in alignment_species]

#     filtered_members = []
#     for member in members:
#         member_species = member.get('species', '').lower().replace(' ', '_')
#         if member_species in alignment_species_lower:
#             filtered_members.append(member)

#     print(f"  📊 Filtered from {len(members)} to {len(filtered_members)} members")
#     print(f"  🎯 Species in filtered set: {len(set(m.get('species') for m in filtered_members))}")

#     return filtered_members

# def run_comprehensive_gene_discovery_enhanced() -> pd.DataFrame:
#     """
#     Enhanced discovery pipeline with alignment-based species filtering.
#     """
#     print("\n" + "="*70)
#     print("🚀 ENHANCED GENE FAMILY DISCOVERY WITH SPECIES FILTERING")
#     print("="*70)

#     tree_result = None

#     # Strategy 1: Gene symbol lookup
#     print(f"\n🎯 Strategy 1: Gene Symbol Lookup ({gene_symbol})")
#     tree_result = get_gene_tree_by_symbol(gene_symbol, reference_species)

#     if tree_result and tree_result.get('members'):
#         print(f"   ✅ Found {len(tree_result['members'])} total members")

#         # Apply species filtering if requested
#         if dataset_scope == "alignment_subset":
#             print(f"\n🔬 Applying species filtering for: {ensembl_alignment}")
#             alignment_species = get_alignment_species_list(ensembl_alignment)

#             if alignment_species:
#                 # Filter members to alignment species
#                 filtered_members = filter_members_by_alignment(tree_result['members'], alignment_species)
#                 tree_result['members'] = filtered_members
#                 tree_result['tree_id'] = f"{tree_result['tree_id']}_filtered_{ensembl_alignment.replace(' ', '_')}"
#             else:
#                 print(f"  🔄 Alignment filtering failed, using manual species limit")
#                 # Fallback: limit to common mammalian species for "44 eutherian mammals EPO"
#                 if "mammals" in ensembl_alignment.lower():
#                     mammal_keywords = ['homo_sapiens', 'mus_musculus', 'rattus_norvegicus',
#                                      'bos_taurus', 'sus_scrofa', 'canis_lupus', 'felis_catus',
#                                      'macaca_mulatta', 'pan_troglodytes', 'gorilla_gorilla']
#                     filtered_members = [m for m in tree_result['members']
#                                       if any(keyword in m.get('species', '').lower()
#                                            for keyword in mammal_keywords)]
#                     tree_result['members'] = filtered_members[:44]  # Limit to ~44
#                     print(f"  🔄 Applied manual mammalian filter: {len(tree_result['members'])} members")

#         else:
#             print(f"\n📊 Using ALL species (no filtering)")
#             print(f"   ⚠️  This will process {len(tree_result['members'])} genes - may take time!")

#     # Fallback strategies (same as before)
#     if not tree_result or not tree_result.get('members'):
#         print(f"\n🧪 Strategy 2: Test with Known Working Tree")
#         tree_result = test_with_known_working_tree()
#         # Apply same filtering logic here if needed

#     if not tree_result or not tree_result.get('members'):
#         print(f"\n🔄 Strategy 3: Alternative Gene Symbols")
#         alternative_genes = ["BRCA2", "COL1A2", "TP53"]
#         for alt_gene in alternative_genes:
#             print(f"  Trying {alt_gene}...")
#             tree_result = get_gene_tree_by_symbol(alt_gene, reference_species)
#             if tree_result and tree_result.get('members'):
#                 print(f"  ✅ Success with {alt_gene}!")
#                 break

#     if not tree_result or not tree_result.get('members'):
#         print("❌ All discovery strategies failed")
#         return pd.DataFrame()

#     members = tree_result['members']
#     tree_id = tree_result['tree_id']

#     print(f"\n✅ Successfully discovered gene family!")
#     print(f"   Tree ID: {tree_id}")
#     print(f"   Members to process: {len(members)}")

#     # Show species distribution
#     species_counts = {}
#     for member in members:
#         species = member.get('species', 'unknown')
#         species_counts[species] = species_counts.get(species, 0) + 1

#     print(f"\n📊 Species Distribution:")
#     for species, count in sorted(species_counts.items())[:10]:  # Show top 10
#         print(f"   {species}: {count} genes")
#     if len(species_counts) > 10:
#         print(f"   ... and {len(species_counts) - 10} more species")

#     # Continue with exon collection (same as before)
#     print(f"\n📊 Collecting exon data for {len(members)} genes...")
#     all_exon_data = []
#     successful_genes = 0

#     for i, member in enumerate(members, 1):
#         gene_id = (member.get('gene_stable_id') or
#                   member.get('protein_stable_id') or
#                   member.get('id') or
#                   member.get('stable_id'))

#         species = (member.get('species') or
#                   member.get('genome'))

#         gene_name = member.get('display_name', member.get('name', 'Unknown'))

#         if not gene_id or not species:
#             print(f"  [{i:2d}/{len(members)}] ⚠️ Skipping: incomplete data")
#             continue

#         print(f"  [{i:2d}/{len(members)}] Processing: {gene_id} ({species})")

#         exons = get_exon_details_for_gene(gene_id, species)
#         if exons:
#             successful_genes += 1
#             for exon_num, exon in enumerate(exons, 1):
#                 all_exon_data.append({
#                     'species': species,
#                     'gene_id': gene_id,
#                     'gene_name': gene_name,
#                     'exon_id': exon.get('id'),
#                     'exon_number_in_gene': exon_num,
#                     'start': exon.get('start'),
#                     'end': exon.get('end'),
#                     'strand': exon.get('strand'),
#                     'length': (exon.get('end', 0) - exon.get('start', 0) + 1) if exon.get('start') and exon.get('end') else 0,
#                     'tree_id': tree_id,
#                     'alignment_used': ensembl_alignment if dataset_scope == "alignment_subset" else "all_species"
#                 })

#     if not all_exon_data:
#         print("❌ No exon data could be collected")
#         return pd.DataFrame()

#     results_df = pd.DataFrame(all_exon_data)

#     print(f"\n🎉 SUCCESS!")
#     print(f"   📊 Total exons: {len(results_df)}")
#     print(f"   🧬 Unique genes: {results_df['gene_id'].nunique()}")
#     print(f"   🌍 Species: {results_df['species'].nunique()}")
#     print(f"   🔬 Alignment: {ensembl_alignment if dataset_scope == 'alignment_subset' else 'All species'}")
#     print("="*70)

#     return results_df

# # Execute the enhanced pipeline
# print("🚀 Starting enhanced gene family discovery...")
# raw_exon_data = run_comprehensive_gene_discovery_enhanced()

# # Display results (same as before)
# if not raw_exon_data.empty:
#     print("\n--- Sample of Successfully Collected Data ---")
#     display(raw_exon_data[['species', 'gene_name', 'exon_number_in_gene', 'start', 'end', 'length']].head(10))

#     print("\n--- Species Distribution ---")
#     species_summary = raw_exon_data.groupby('species').agg({
#         'gene_id': 'nunique',
#         'exon_id': 'count',
#         'length': 'mean'
#     }).rename(columns={'gene_id': 'genes', 'exon_id': 'exons', 'length': 'avg_exon_length'})
#     species_summary['avg_exon_length'] = species_summary['avg_exon_length'].round(1)
#     display(species_summary.head(15))

🚀 Starting enhanced gene family discovery...

🚀 ENHANCED GENE FAMILY DISCOVERY WITH SPECIES FILTERING

🎯 Strategy 1: Gene Symbol Lookup (COL1A1)
🔍 Looking up gene tree for COL1A1 in homo_sapiens
🔍 API Response Debug for: https://rest.ensembl.org/genetree/member/symbol/homo_sapiens/COL1A1
   Top-level keys: ['rooted', 'type', 'id', 'tree']
   rooted: int = 1
   type: str = gene tree
   id: str = ENSGT00940000156584
   tree: Dict with keys ['taxonomy', 'children', 'confidence', 'events', 'branch_length']

🔍 Detailed tree structure analysis:
Node keys: ['taxonomy', 'children', 'confidence', 'events', 'branch_length']
children: List with 2 children
  Child 0:
    Node keys: ['events', 'confidence', 'children', 'taxonomy', 'branch_length']
    children: List with 2 children
      Child 0:
        ... (max depth reached)
      Child 1:
        ... (max depth reached)
  Child 1:
    Node keys: ['branch_length', 'children', 'taxonomy', 'confidence', 'events']
    children: List with 2 children

,species,gene_name,exon_number_in_gene,start,end,length
0,Dasypus novemcinctus,BRCA2-201,1,90758,90821,64
1,Dasypus novemcinctus,BRCA2-201,2,89076,89327,252
2,Dasypus novemcinctus,BRCA2-201,3,83184,83292,109
3,Dasypus novemcinctus,BRCA2-201,4,82634,82680,47
4,Dasypus novemcinctus,BRCA2-201,5,82489,82529,41
5,Dasypus novemcinctus,BRCA2-201,6,76011,76122,112
6,Dasypus novemcinctus,BRCA2-201,7,73623,73672,50
7,Dasypus novemcinctus,BRCA2-201,8,73079,73187,109
8,Dasypus novemcinctus,BRCA2-201,9,68862,69977,1116
9,Dasypus novemcinctus,BRCA2-201,10,62152,67149,4998



--- Species Distribution ---


,genes,exons,avg_exon_length
species,,,
Acanthochromis polyacanthus,1,27,308.9
Ailuropoda melanoleuca reference isolate,1,26,400.7
Amphilophus citrinellus,1,15,171.0
Amphiprion ocellaris ecotype Okinawa,1,13,514.2
Anabas testudineus reference strain,1,18,289.7
Anas platyrhynchos platyrhynchos,1,26,390.7
Anolis carolinensis reference strain,1,22,368.0
Aotus nancymaae,1,26,392.9
Aquila chrysaetos chrysaetos,1,25,408.6


In [ ]:
# # ===== Cell 5 =====
# # Solve the "split exon problem" by mapping all exons to reference species structure

# def map_exons_to_reference(target_exons: List[Dict], reference_exons: List[Dict]) -> List[Dict]:
#     """
#     Maps target species exons to reference species exon numbering.
#     Handles split exons (5-6) and insertions (21a, 21b).
#     This solves the core "split exon problem" described in your goals.
#     """
#     mapped_results = []
#     last_mapped_ref = 0
#     insertion_counter = 0

#     # Sort both by genomic position
#     target_sorted = sorted(target_exons, key=lambda x: x.get('start', 0))
#     ref_sorted = sorted(reference_exons, key=lambda x: x.get('start', 0))

#     # Add reference numbers for easier lookup
#     for i, ref_exon in enumerate(ref_sorted):
#         ref_exon['ref_number'] = i + 1

#     for target_exon in target_sorted:
#         target_start = target_exon.get('start', 0)
#         target_end = target_exon.get('end', 0)

#         # Find overlapping reference exons
#         overlapping_refs = []
#         for ref_exon in ref_sorted:
#             ref_start = ref_exon.get('start', 0)
#             ref_end = ref_exon.get('end', 0)

#             # Check for overlap
#             if max(target_start, ref_start) <= min(target_end, ref_end):
#                 overlapping_refs.append(ref_exon['ref_number'])

#         # Determine standardized number
#         if overlapping_refs:
#             if len(overlapping_refs) == 1:
#                 std_number = str(overlapping_refs[0])
#             else:
#                 std_number = f"{min(overlapping_refs)}-{max(overlapping_refs)}"
#             last_mapped_ref = max(overlapping_refs)
#             insertion_counter = 0
#         else:
#             # This is an insertion relative to reference
#             insertion_counter += 1
#             std_number = f"{last_mapped_ref}{chr(96 + insertion_counter)}"  # 21a, 21b, etc.

#         mapped_results.append({
#             **target_exon,
#             'standardized_exon_number': std_number,
#             'mapping_type': 'mapped' if overlapping_refs else 'insertion'
#         })

#     return mapped_results

# # Perform the standardization
# if not raw_exon_data.empty:
#     print("\n" + "="*60)
#     print("🎯 STARTING EXON STANDARDIZATION")
#     print("="*60)

#     # Get reference exons
#     ref_exons = raw_exon_data[raw_exon_data['species'] == reference_species].to_dict('records')

#     if not ref_exons:
#         print(f"❌ No exon data found for reference species: {reference_species}")
#     else:
#         print(f"📐 Using {len(ref_exons)} exons from {reference_species} as reference")

#         all_standardized = []

#         # Process each species separately
#         for species, species_group in raw_exon_data.groupby('species'):
#             print(f"  🔄 Standardizing: {species}")

#             for gene_id, gene_group in species_group.groupby('gene_id'):
#                 target_exons = gene_group.to_dict('records')
#                 standardized = map_exons_to_reference(target_exons, ref_exons)
#                 all_standardized.extend(standardized)

#         # Create final standardized DataFrame
#         standardized_df = pd.DataFrame(all_standardized)

#         print(f"\n✅ Standardization Complete!")
#         print(f"   Total standardized exons: {len(standardized_df)}")
#         print(f"   Split exons detected: {len(standardized_df[standardized_df['standardized_exon_number'].str.contains('-')])}")
#         print(f"   Insertion exons detected: {len(standardized_df[standardized_df['mapping_type'] == 'insertion'])}")

#         # Save results
#         output_filename = f"{gene_tree_id}_{tissue_of_interest}_standardized_exons.csv"
#         output_path = output_dir / output_filename
#         standardized_df.to_csv(output_path, index=False)

#         print(f"\n💾 Results saved to: {output_path}")
#         print("="*60)

#         # Display sample results
#         print("\n--- Sample of Standardized Results ---")
#         display(standardized_df[['species', 'gene_name', 'exon_number_in_gene',
#                                'standardized_exon_number', 'mapping_type', 'start', 'end']].head(15))

#         print(f"\n--- Final Dataset Summary ---")
#         print(f"✅ Ready for PhASTM pipeline: {len(standardized_df)} standardized exons")
#         print(f"✅ Species coverage: {standardized_df['species'].nunique()} species")
#         print(f"✅ Gene coverage: {standardized_df['gene_id'].nunique()} orthologous genes")

# else:
#     print("❌ No data available for standardization")